<a href="https://colab.research.google.com/github/StevenBryceLee/DS-Unit-2-Linear-Models/blob/master/module2-regression-2/Copy_of_LS_DS_212_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [102]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [103]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [104]:
#Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
from sklearn.model_selection import train_test_split

In [105]:
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [106]:
#Get the last day of may and subset the training and test data
date_may = '2016-05-31'
#Before the last day in may
df_train = df[df['created'] <= date_may].reset_index().drop('index',axis = 1)
#First day of June onward
df_test = df[df['created'] > date_may].reset_index().drop('index',axis = 1)

In [107]:
df_train.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2.0,4,2016-04-19 04:24:47,,West 18th Street,40.7429,-74.0028,7995,350 West 18th Street,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,2,2016-04-27 03:19:56,Stunning unit with a great location and lots o...,West 107th Street,40.8012,-73.9660,3600,210 West 107th Street,low,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [108]:
df_test.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-06-03 03:21:22,Check out this one bedroom apartment in a grea...,W. 173rd Street,40.8448,-73.9396,1675,644 W. 173rd Street,low,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-06-01 03:11:01,Spacious 1-Bedroom to fit King-sized bed comfo...,East 56th St..,40.7584,-73.9648,3050,315 East 56th St..,low,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2.0,4,2016-06-07 04:39:56,SPRAWLING 2 BEDROOM FOUND! ENJOY THE LUXURY OF...,W 18 St.,40.7391,-73.9936,7400,30 W 18 St.,medium,1,1,1,1,1,1,0,0,1,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0


In [109]:
import re

In [110]:
#Working correctly
#Engineer at least two new features. (See below for explanation & ideas.)
#This feature comes from finding out if the apartment is 'new' by checking the description

df_train['new'] = [1 if 'new' in string else 0 for string in df_train['description'].fillna(' ')]

#This feature comes from finding out if the address is a numbered street or if it has a name
df_train['named_address'] = [0 if bool(re.search(r'\d', string)) else 1
                             for string in df_train['display_address'].fillna('1')]

In [111]:
df_train[['description','new']][0:5]

,description,new
0,"Top Top West Village location, beautiful Pre-w...",0
1,Building Amenities - Garage - Garden - fitness...,0
2,Beautifully renovated 3 bedroom flex 4 bedroom...,0
3,,0
4,Stunning unit with a great location and lots o...,0


In [112]:
df_train[['display_address','named_address']][0:5]

,display_address,named_address
0,W 13 Street,0
1,East 49th Street,0
2,West 143rd Street,0
3,West 18th Street,0
4,West 107th Street,0


In [113]:
# Fit a linear regression model with at least two features.
from sklearn.linear_model import LinearRegression
#Instantiate the model
lr = LinearRegression()
#Get regressors, drop non numeric values and target
X = df_train.select_dtypes(['number']).drop('price',axis = 1)
#Get target
y = df_train[['price']]
lr.fit(X,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [114]:
#This is what that is doing
df_train.select_dtypes(['number']).head()

,bathrooms,bedrooms,latitude,longitude,price,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,new,named_address
0,1.0,1,40.7388,-74.0018,2850,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,1,40.7539,-73.9677,3275,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,4,40.8241,-73.9493,3350,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2.0,4,40.7429,-74.0028,7995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,2,40.8012,-73.9660,3600,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [115]:
#Get the model's coefficients and intercept.
print('coefficients:\t',lr.coef_)
print('intercept:\t',lr.intercept_)

coefficients:	 [[ 1.76566333e+03  4.80042802e+02  1.50699062e+03 -1.37201765e+04
   1.49085571e+02 -4.39930658e+01 -2.06994108e+02  1.04648684e+02
   4.81550891e+02  5.03182324e+01 -1.68131602e+02 -2.58143890e+02
   1.44207197e+02 -6.75637390e+01  4.99337872e+02 -1.71631621e+02
  -1.13839920e+02  2.39502424e+02 -3.45697022e+02 -5.13567208e+01
   5.46495026e+01 -1.35880253e+02  1.83394171e+02  1.14897712e+02
   1.17716561e+02 -4.76287349e+01  2.03137433e+02 -9.21234501e+01
   1.21967089e-01 -4.82309875e+01]]
intercept:	 [-1075837.58612203]


In [116]:
#coefficients are the beta values for each feature
#Intercept is the y value when all x values == 0
print(len(lr.coef_[0]))
print(len(X.columns))

30
30


In [117]:

df_test['new'] = [1 if 'new' in string else 0 for string in df_test['description'].fillna(' ')]
df_test['named_address'] = [0 if bool(re.search(r'\d', string)) else 1
                             for string in df_test['display_address'].fillna('1')]
X_test = df_test.select_dtypes(['number']).drop('price',axis = 1)

In [118]:
#Get regression metrics RMSE, MAE, and  R2 , for both the train and test data.
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import r2_score as R2
#get predicted and actual values for training
y_true_train = y
y_pred_train = lr.predict(X)
#get predicted and actual values for test
y_true_test = df_test[['price']]
X_test = df_test.select_dtypes(['number']).drop('price',axis = 1)
y_pred_test =lr.predict(X_test)
#Get MSE on train, test
MSE_list = [MSE(y_true_train,y_pred_train,squared = False),
            MSE(y_true_test,y_pred_test,squared = False)]
print('Root Mean Squared Error:\n\t{}\n\t{}'.format(MSE_list[0],MSE_list[1]))

#Get MAE on train, test
MAE_list = [MAE(y_true_train,y_pred_train),
            MAE(y_true_test,y_pred_test)]
print('\nMean Absolute Error:\n\t{}\n\t{}'.format(MAE_list[0],MAE_list[1]))

#Get r2_score on train, test
R2_list = [R2(y_true_train,y_pred_train),
           R2(y_true_test,y_pred_test)]
print('\nR2 score:\n\t{}\n\t{}'.format(R2_list[0],R2_list[1]))

Root Mean Squared Error:
	1089.9420496626897
	1078.767838964229

Mean Absolute Error:
	692.1728664814294
	701.1384657160856

R2 score:
	0.6174451480339112
	0.6254925800157735


In [119]:
#What's the best test MAE you can get? 
#Hmm. I bet it would help if I at least got some categorical features into the model
#Stretch goal is to encode some realtor-speak into a number
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [120]:
df.created = pd.to_datetime(df['created']).dt.date
df.created

0        2016-06-24
1        2016-06-12
2        2016-04-17
3        2016-04-18
4        2016-04-28
            ...    
49347    2016-06-02
49348    2016-04-04
49349    2016-04-16
49350    2016-04-08
49351    2016-04-12
Name: created, Length: 48817, dtype: object

In [121]:
from datetime import datetime
date_from_start = pd.to_datetime(df['created']).dt.date - pd.to_datetime(df['created']).dt.date.min()

In [122]:
date_from_start

0       84 days
1       72 days
2       16 days
3       17 days
4       27 days
          ...  
49347   62 days
49348    3 days
49349   15 days
49350    7 days
49351   11 days
Name: created, Length: 48817, dtype: timedelta64[ns]

In [123]:
type(date_from_start[0])

pandas._libs.tslibs.timedeltas.Timedelta

In [124]:
df['day_count'] = date_from_start.apply(lambda x: x.days)

In [125]:
df['day_count']

0        84
1        72
2        16
3        17
4        27
         ..
49347    62
49348     3
49349    15
49350     7
49351    11
Name: day_count, Length: 48817, dtype: int64

In [126]:
#Dropping created now that we have a working int column
df.drop('created',axis = 1,inplace=True)

In [127]:
df.head()

,bathrooms,bedrooms,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,day_count
0,1.5,3,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,84
1,1.0,2,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,72
2,1.0,1,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16
3,1.0,1,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17
4,1.0,4,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,27


In [128]:
df.at[0,'description']

"A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy These Following Apartment Features As You Rent Here? Modern Designed Bathroom w/ a Deep Spa Soaking Tub? Room to Room AC/Heat? Real Oak Hardwood Floors? Rain Forest Shower Head? SS steel Appliances w/ Chef Gas Cook Oven & LG Fridge? washer /dryer in the apt? Cable Internet Ready? Granite Counter Top Kitchen w/ lot of cabinet storage spaceIt's Just A Few blocks To L Train<br /><br />Don't miss out!<br /><br />We have several great apartments in the immediate area.<br /><br />For additional information 687-878-2229<p><a  website_redacted "

In [129]:
#Wait. This database just has people's phone numbers. Nice
#A pretty rough metric would just be how long the description is. I wonder if that helps
#df['description_length'] = len(df['description'].strip(['<br />','<p>']))

In [130]:
description_length= df['description'].fillna(' ').apply(lambda x: len(x.strip('<br />')))
description_length

0         587
1           0
2         690
3         491
4         478
         ... 
49347     786
49348    1125
49349     669
49350     733
49351     798
Name: description, Length: 48817, dtype: int64

In [131]:
df['description_length'] = description_length

In [132]:
df.drop('description',axis = 1, inplace= True)
df.head()

,bathrooms,bedrooms,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,day_count,description_length
0,1.5,3,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,84,587
1,1.0,2,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,72,0
2,1.0,1,W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,690
3,1.0,1,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17,491
4,1.0,4,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,27,478


In [133]:
#Making progress
#I think I need to drop address, since latitude and longitude should capture the same info
df.drop(['display_address','street_address'],axis = 1, inplace=True)
df.head()

,bathrooms,bedrooms,latitude,longitude,price,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,day_count,description_length
0,1.5,3,40.7145,-73.9425,3000,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,84,587
1,1.0,2,40.7947,-73.9667,5465,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,72,0
2,1.0,1,40.7388,-74.0018,2850,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,690
3,1.0,1,40.7539,-73.9677,3275,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17,491
4,1.0,4,40.8241,-73.9493,3350,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,27,478


In [134]:
df.head()

,bathrooms,bedrooms,latitude,longitude,price,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,day_count,description_length
0,1.5,3,40.7145,-73.9425,3000,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,84,587
1,1.0,2,40.7947,-73.9667,5465,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,72,0
2,1.0,1,40.7388,-74.0018,2850,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,690
3,1.0,1,40.7539,-73.9677,3275,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17,491
4,1.0,4,40.8241,-73.9493,3350,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,27,478


In [135]:
#Interest level can be one hot encoded
df_interest = pd.get_dummies(df,prefix = 'il', columns = ['interest_level'],
                                  #Prevent redundancy by dropping first variable
                                  drop_first=True,
                                  )
df_interest

,bathrooms,bedrooms,latitude,longitude,price,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,day_count,description_length,il_low,il_medium
0,1.5,3,40.7145,-73.9425,3000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,84,587,0,1
1,1.0,2,40.7947,-73.9667,5465,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,72,0,1,0
2,1.0,1,40.7388,-74.0018,2850,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,690,0,0
3,1.0,1,40.7539,-73.9677,3275,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17,491,1,0
4,1.0,4,40.8241,-73.9493,3350,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,27,478,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49347,1.0,2,40.7426,-73.9790,3200,1,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,62,786,0,1
49348,1.0,1,40.7102,-74.0163,3950,1,1,0,1,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,3,1125,1,0
49349,1.0,1,40.7601,-73.9900,2595,1,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,669,1,0
49350,1.0,0,40.7066,-74.0101,3350,1,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,733,1,0


In [139]:
stats = df.day_count.describe()
stats

count    48817.000000
mean        45.025626
std         25.859673
min          0.000000
25%         22.000000
50%         45.000000
75%         68.000000
max         89.000000
Name: day_count, dtype: float64

In [140]:
stats.loc['75%']

68.0

In [144]:
#Finally, everything is a number. Rerunning previous code to see if that helped
#Retrying analysis with 25% test data
df_train = df_interest[df_interest['day_count'] <= stats.loc['75%']].reset_index().drop('index',axis = 1)

df_test = df_interest[df_interest['day_count'] > stats.loc['75%']].reset_index().drop('index',axis = 1)

In [145]:
# Fit a linear regression model with at least two features.
from sklearn.linear_model import LinearRegression
#Instantiate the model
lr = LinearRegression()
#Get regressors, drop non numeric values and target
X = df_train.drop('price',axis = 1)
#Get target
y = df_train[['price']]
lr.fit(X,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [146]:
#Get regression metrics RMSE, MAE, and  R2 , for both the train and test data.
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import r2_score as R2
#get predicted and actual values for training
y_true_train = y
y_pred_train = lr.predict(X)
#get predicted and actual values for test
y_true_test = df_test[['price']]
X_test = df_test.select_dtypes(['number']).drop('price',axis = 1)
y_pred_test =lr.predict(X_test)
#Get MSE on train, test
MSE_list = [MSE(y_true_train,y_pred_train,squared = False),
            MSE(y_true_test,y_pred_test,squared = False)]
print('Root Mean Squared Error:\n\t{}\n\t{}'.format(MSE_list[0],MSE_list[1]))

#Get MAE on train, test
MAE_list = [MAE(y_true_train,y_pred_train),
            MAE(y_true_test,y_pred_test)]
print('\nMean Absolute Error:\n\t{}\n\t{}'.format(MAE_list[0],MAE_list[1]))

#Get r2_score on train, test
R2_list = [R2(y_true_train,y_pred_train),
           R2(y_true_test,y_pred_test)]
print('\nR2 score:\n\t{}\n\t{}'.format(R2_list[0],R2_list[1]))

Root Mean Squared Error:
	1056.248257816239
	1052.9146264470146

Mean Absolute Error:
	672.8955421308641
	683.2594137675759

R2 score:
	0.6410062093300919
	0.6425023092569134


Previous values

Root Mean Squared Error:

	1089.9420496626897
	1078.767838964229

Mean Absolute Error:

	692.1728664814294
	701.1384657160856

R2 score:

	0.6174451480339112
	0.6254925800157735

We improved MAE by about 20, which is pretty good. Could probably improve more using PCA and scaling the data


In [149]:
#Original notebook
#https://colab.research.google.com/drive/1YfNZhL7chtN1sMtq2GOW0HI2nuuoijMj?usp=sharing